<a href="https://colab.research.google.com/github/kazarezova/bioinformatics_for_NGS_exome/blob/main/kazarezova_ngs_exome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 
Используйте современный germline variant caller (в качестве основного будем
считать GATK, но вы можете выбрать другой) для получения стартового vcf файла.
Оцените полученные результаты:
- Количество синонимичных и несинонимичных замен
- ti/tv (отношение транзиций к трансверсиям) в кодирующих участках

In [ ]:
for i in ['chd_child.bam',
          'chd_child.bam.bai',
          'GRCh38_full_analysis_set_plus_decoy_hla.fa.gz',
          'GRCh38_full_analysis_set_plus_decoy_hla.fa.fai',
          'GRCh38_full_analysis_set_plus_decoy_hla.dict']:
          !wget https://makarich.fbb.msu.ru/enabieva/ngs2022/{i}

In [2]:
!gunzip GRCh38_full_analysis_set_plus_decoy_hla.fa.gz

### Variant caller

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install -c bioconda gatk4

In [ ]:
!gatk --java-options "-Xmx4g" HaplotypeCaller -R GRCh38_full_analysis_set_plus_decoy_hla.fa -I chd_child.bam -O chd_child.vcf.gz

### Аннотируем варианты и оставляем только кодирующие мутации

In [ ]:
!wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
!unzip snpEff_latest_core.zip

In [7]:
!gunzip chd_child.vcf.gz

In [ ]:
!java -Xmx8g -jar snpEff/snpEff.jar GRCh38.99 chd_child.vcf > chd_child.ann.vcf

In [2]:
!egrep '#|synonymous|missense|stop_gained' chd_child.ann.vcf > chd_child.ann.coding.vcf

### Количество синонимичных и несинонимичных замен

In [3]:
!egrep '#|synonymous' chd_child.ann.coding.vcf | wc -l

14202


In [4]:
!egrep '#|missense|stop_gained' chd_child.ann.coding.vcf | wc -l

13716


### ti/tv (отношение транзиций к трансверсиям) в кодирующих участках



In [5]:
# https://pcingola.github.io/SnpEff/ss_tstv/
!java -jar snpEff/SnpSift.jar tstv chd_child.ann.coding.vcf

00:00:00 Analysing 'chd_child.ann.coding.vcf'
...................................................................................................
100	....................................................................................................
200	....................................................................................................
300	....................................................................................................
400	....................................................................................................
500	....................................................................................................
600	....................................................................................................
700	....................................................................................................
800	....................................................................................................
900	..........

# 2
Проведите фильтрацию полученных вариантов, используя либо
специализированные программы, либо “жесткую” фильтрацию (при которой вы
задаете пороги для разных свойств вариантов). Обоснуйте ваш выбор параметров
(следование рекомендациям - ок, но напишите, каким). Не забывайте, что
параметры могут быть разными для однонуклеотидных вариантов (SNPs) и
инделов.
Оцените количество синонимичных и несинонимичных замен и ts/tv после
фильтрации.
Прокомментируйте эффект фильтрации.


### Фильтрация SNP

In [ ]:
# https://gatk.broadinstitute.org/hc/en-us/articles/360035531112--How-to-Filter-variants-either-with-VQSR-or-by-hard-filtering#2
!gatk SelectVariants -V chd_child.vcf -select-type SNP -O chd_child.snps.vcf.gz

In [ ]:
# https://gatk.broadinstitute.org/hc/en-us/articles/360035531112--How-to-Filter-variants-either-with-VQSR-or-by-hard-filtering
!gatk VariantFiltration \
    -V chd_child.snps.vcf.gz \
    -filter "QD < 2.0" --filter-name "QD2" \
    -filter "QUAL < 30.0" --filter-name "QUAL30" \
    -filter "SOR > 3.0" --filter-name "SOR3" \
    -filter "FS > 60.0" --filter-name "FS60" \
    -filter "MQ < 40.0" --filter-name "MQ40" \
    -filter "MQRankSum < -12.5" --filter-name "MQRankSum-12.5" \
    -filter "ReadPosRankSum < -8.0" --filter-name "ReadPosRankSum-8" \
    -O chd_child.snps_filtered.vcf.gz

In [ ]:
!gunzip chd_child.snps_filtered.vcf.gz
!java -Xmx8g -jar snpEff/snpEff.jar GRCh38.99 chd_child.snps_filtered.vcf > chd_child.snps_filtered.ann.vcf
!egrep '#|synonymous|missense|stop_gained' chd_child.snps_filtered.ann.vcf > chd_child.snps_filtered.ann.coding.vcf

In [18]:
!grep 'PASS' chd_child.snps_filtered.ann.coding.vcf > chd_child.snps_filtered_pass.ann.coding.vcf

### Количество синонимичных и несинонимичных замен после фильтрации SNP

In [19]:
!egrep '#|synonymous' chd_child.snps_filtered_pass.ann.coding.vcf | wc -l

10492


In [20]:
!egrep '#|missense|stop_gained' chd_child.snps_filtered_pass.ann.coding.vcf | wc -l

9927


### ti/tv в кодирующих участках после фильтрации

In [21]:
!java -jar snpEff/SnpSift.jar tstv chd_child.snps_filtered_pass.ann.coding.vcf

00:00:00 Analysing 'chd_child.snps_filtered_pass.ann.coding.vcf'
...................................................................................................
100	....................................................................................................
200	....................................................................................................
300	....................................................................................................
400	....................................................................................................
500	....................................................................................................
600	....................................................................................................
700	....................................................................................................
800	................................................................................................

# 3 
Напишите, сколько в вашем окончательном списке:
- Protein truncating variants (инделов со сдвигом рамки считывания,
nonsense-вариантов)
- Гомозиготных вариантов, изменяющих последовательность белка
- Приведите их список

### Protein truncating variants

In [22]:
# nonsense-варианты

!egrep '#|stop_gained' chd_child.snps_filtered_pass.ann.coding.vcf | wc -l

89


In [39]:
!egrep '#|stop_gained' chd_child.snps_filtered_pass.ann.coding.vcf

chr1	40307478	.	G	A	142.64	PASS	AC=1;AF=0.500;AN=2;BaseQRankSum=-0.987;DP=11;ExcessHet=0.0000;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=60.00;MQRankSum=0.000;QD=14.26;ReadPosRankSum=0.204;SOR=0.892;ANN=A|stop_gained|HIGH|COL9A2|ENSG00000049089|transcript|ENST00000372748.8|protein_coding|19/32|c.976C>T|p.Gln326*|1065/2852|976/2070|326/689||,A|upstream_gene_variant|MODIFIER|COL9A2|ENSG00000049089|transcript|ENST00000427563.1|processed_transcript||n.-2977C>T|||||2977|,A|upstream_gene_variant|MODIFIER|COL9A2|ENSG00000049089|transcript|ENST00000466267.1|processed_transcript||n.-2481C>T|||||2481|,A|downstream_gene_variant|MODIFIER|COL9A2|ENSG00000049089|transcript|ENST00000461118.6|retained_intron||n.*4752C>T|||||4752|,A|downstream_gene_variant|MODIFIER|COL9A2|ENSG00000049089|transcript|ENST00000372736.3|protein_coding||c.*4193C>T|||||4193|WARNING_TRANSCRIPT_NO_STOP_CODON,A|downstream_gene_variant|MODIFIER|COL9A2|ENSG00000049089|transcript|ENST00000488463.6|processed_transcript||n.*2633C>T|||||2633|,A

In [ ]:
# инделы со сдвигом рамки считывания (т.е. любые кодирующие инделы, кроме синонимичных)

!gatk SelectVariants -V chd_child.vcf -select-type INDEL -O chd_child.indels.vcf.gz

!gatk VariantFiltration -V chd_child.indels.vcf.gz -filter "QD < 2.0" --filter-name "QD2" -filter "QUAL < 30.0" --filter-name "QUAL30" -filter "FS > 200.0" --filter-name "FS200" -filter "ReadPosRankSum < -20.0" --filter-name "ReadPosRankSum-20" -O chd_child.indels_filtered.vcf.gz

!gunzip chd_child.indels_filtered.vcf.gz
!java -Xmx8g -jar snpEff/snpEff.jar GRCh38.99 chd_child.indels_filtered.vcf > chd_child.indels_filtered.ann.vcf
!egrep '#|missense|stop_gained' chd_child.indels_filtered.ann.vcf > chd_child.indels_filtered.ann.coding.vcf

In [34]:
!grep 'PASS' chd_child.indels_filtered.ann.coding.vcf | wc -l

5


In [38]:
!grep 'PASS' chd_child.indels_filtered.ann.coding.vcf

chr3	149520808	.	C	CTTAA	490.6	PASS	AC=1;AF=0.500;AN=2;BaseQRankSum=1.697;DP=21;ExcessHet=0.0000;FS=2.092;MLEAC=1;MLEAF=0.500;MQ=61.02;MQRankSum=1.209;QD=23.36;ReadPosRankSum=-0.218;SOR=1.371;ANN=CTTAA|frameshift_variant&stop_gained|HIGH|WWTR1|ENSG00000018408|transcript|ENST00000360632.7|protein_coding|7/7|c.1199_1200insTTAA|p.Ter401fs|1539/2103|1199/1203|400/400||,CTTAA|frameshift_variant&stop_gained|HIGH|WWTR1|ENSG00000018408|transcript|ENST00000465804.5|protein_coding|8/8|c.1199_1200insTTAA|p.Ter401fs|1456/5030|1199/1203|400/400||,CTTAA|frameshift_variant&stop_gained|HIGH|WWTR1|ENSG00000018408|transcript|ENST00000467467.5|protein_coding|7/7|c.1199_1200insTTAA|p.Ter401fs|1404/1603|1199/1203|400/400||,CTTAA|3_prime_UTR_variant|MODIFIER|WWTR1|ENSG00000018408|transcript|ENST00000471586.5|nonsense_mediated_decay|4/4|n.*346_*347insTTAA|||||7079|,CTTAA|3_prime_UTR_variant|MODIFIER|WWTR1|ENSG00000018408|transcript|ENST00000494754.1|nonsense_mediated_decay|4/4|n.*390_*391insTTAA|||||7123|,CT

### Гомозиготные варианты, изменяющие последовательность белка

In [ ]:
# референсный геном для GenotypeGVCFs
!wget https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta
!wget https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta.fai
!wget https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dict

In [ ]:
!gatk IndexFeatureFile -I chd_child.snps_filtered.ann.coding.vcf

In [ ]:
# гомозиготные варианты https://gatk.broadinstitute.org/hc/en-us/articles/360042914991-GenotypeGVCFs
!gatk --java-options "-Xmx4g" GenotypeGVCFs \
   -R Homo_sapiens_assembly38.fasta \
   -V chd_child.snps_filtered.ann.coding.vcf \
   -O homo.vcf.gz --include-non-variant-sites

In [66]:
!gunzip homo.vcf.gz

In [72]:
# изменяющие последовательность
!egrep '#|missense' homo.vcf | wc -l

3410
